## Snakemake for Converting GMET Forcing to Snakemake ##

Generate the snakemake rules used in the conversion of gmet forcing to summa input forcing



### Prep GMET files for processing ###

In [21]:
%%writefile ../rules/gmet_file_prep.smk

# This Snakemake file prepares the GMET data for use in 
from pathlib import Path
import sys

sys.path.append('../')
from scripts import gmet_to_summa_utils as gts_utils

# Resolve paths from the configuration file
config = gts_utils.resolve_paths(config)

#Set the list of forcing files to process
# Read all forcing files and create a list based on the output directory (i.e. ens/filename.nc)
ensemble_list, file_path_list = gts_utils.build_ensemble_list(config['gmet_forcing_dir'])

#This first rule establishes the output files that will be created
rule gmet_file_prep:
    input:
        expand(Path(config['gmet_tmp_forcing_dir'],"{forcing_file}.nc"), forcing_file=file_path_list)

#Add greogrian calendar to the time variable, needed for easymore  
rule add_gregorian_to_nc:
    input:  
        input_forcing = Path(config['gmet_forcing_dir'],"{id}.nc")
    output: 
        output_forcing = temp(Path(config['gmet_tmp_forcing_dir'],"{id}_greg.nc"))
    group:
        "gmet_to_summa"
    shell: 
        'ncatted -a "calendar,time,o,c,"gregorian"" {input.input_forcing} {output.output_forcing}'

#Process temperature data to create t_max and t_min
rule add_t_max_and_t_min:
    input: 
        input_file = Path(config['gmet_tmp_forcing_dir'],"{id}_greg.nc")
    output:
        temp = temp(Path(config['gmet_tmp_forcing_dir'],"{ens}","{id}_temp.nc")),
        output_file = Path(config['gmet_tmp_forcing_dir'],"{ens}","{id}.nc")
    group:
        "gmet_to_summa"
    shell:
        """
        ncap2 -s "t_max = t_mean+0.5+t_range" -A {input.input_file} {output.temp};
        ncap2 -s "t_min = t_mean+0.5-t_range" -A {output.temp};
        ncatted -O -a long_name,t_max,o,c,"estimated daily maximum temperature" {output.temp};
        ncatted -O -a long_name,t_min,o,c,"estimated daily minimum temperature" {output.temp};
        cp {output.temp} {output.output_file}
        """

Overwriting ../rules/gmet_file_prep.smk


### Run snakemake file

In [22]:
! snakemake --unlock -s ../rules/gmet_file_prep.smk --configfile ../config/gmet_to_summa_test_case_config.yaml
! snakemake -s ../rules/gmet_file_prep.smk -c 8 --configfile ../config/gmet_to_summa_test_case_config.yaml 

Unlocking working directory.
Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 8
Rules claiming more threads will be scaled down.
Job stats:
job                    count    min threads    max threads
-------------------  -------  -------------  -------------
add_gregorian_to_nc        3              1              1
add_t_max_and_t_min        3              1              1
gmet_file_prep             1              1              1
total                      7              1              1

Select jobs to execute...

[Mon Aug 21 13:08:35 2023]
rule add_gregorian_to_nc:
    input: ../../test_data/gmet/gmet.01d.2021.001.nc
    output: ../../test_data/working_dir/gmet_tmp/gmet.01d.2021.001_greg.nc
    jobid: 6
    reason: Missing output files: ../../test_data/working_dir/gmet_tmp/gmet.01d.2021.001_greg.nc
    wildcards: id=gmet.01d.2021.001
    resources: tmpdir=/var/folders/cf/h3xm_pj57nxdnm6lk83rzxt9jt0gpj/T


[Mon Aug 21 13:08:35 2023]
rule add_gregorian_to_nc:
    input: 

## Remap GMET to SHP ##

In [23]:
%%writefile ../rules/remap_gmet_to_shp.smk

# Import needed packages
from pathlib import Path
import sys
sys.path.append('../')
from scripts import remap_forcing_to_shp
from scripts import gmet_to_summa_utils as gts_utils

# Resolve paths from the configuration file
config = gts_utils.resolve_paths(config)

# Read all forcing files and create a list based on the output directory (i.e. ens/filename.nc)
ensemble_list, file_path_list = gts_utils.build_ensemble_list(config['gmet_forcing_dir'])

rule remap_gmet_to_shp:
    input:
        expand(Path(config['easymore_output_dir'],"{file}.nc"), file=file_path_list)

# Define rule to run file remapping when remap file exists
rule create_remap_file:
    input:
        input_forcing_files = expand(Path(config['gmet_tmp_forcing_dir'],"{forcing_file}.nc"), forcing_file=file_path_list),
        input_shp = config['catchment_shp']
    output:
        remap_csv = config['remap_file']
    run:
        remap_forcing_to_shp.remap_with_easymore(config, input.input_forcing_files[0],input.input_shp, output.remap_csv, only_create_remap_csv=True)

# Define rule to run file remapping when remap file exists
rule remap_with_easymore:
    input:
        input_forcing = Path(config['gmet_tmp_forcing_dir'],"{id}.nc"),
        input_shp = config['catchment_shp'],
        remap_csv = config['remap_file'],
    output:
        output_forcing = Path(config['easymore_output_dir'],"{id}.nc")
    params:
        file_path = "{id}"
    run:
        remap_forcing_to_shp.remap_with_easymore(config, input.input_forcing,input.input_shp,input.remap_csv,only_create_remap_csv=False,file_path=params.file_path)





Overwriting ../rules/remap_gmet_to_shp.smk


In [24]:
! snakemake --unlock -s ../rules/remap_gmet_to_shp.smk  --configfile  ../config/gmet_to_summa_test_case_config.yaml
! snakemake -s ../rules/remap_gmet_to_shp.smk -c 8 --configfile  ../config/gmet_to_summa_test_case_config.yaml

Unlocking working directory.
Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 8
Rules claiming more threads will be scaled down.
Job stats:
job                    count    min threads    max threads
-------------------  -------  -------------  -------------
create_remap_file          1              1              1
remap_gmet_to_shp          1              1              1
remap_with_easymore        3              1              1
total                      5              1              1

Select jobs to execute...

[Mon Aug 21 13:08:45 2023]
rule create_remap_file:
    input: ../../test_data/working_dir/gmet_tmp/003/gmet.01d.2021.003.nc, ../../test_data/working_dir/gmet_tmp/002/gmet.01d.2021.002.nc, ../../test_data/working_dir/gmet_tmp/001/gmet.01d.2021.001.nc, ../../test_data/shp/tuolumne_elev_band.gpkg
    output: ../../test_data/working_dir/easymore/temp/test_case_remapping.csv
    jobid: 2
    reason: Missing output files: ../../test_data/working_dir/easymore/temp/te

## Prepare gmet files for metsim ##

In [25]:
%%writefile ../rules/metsim_file_prep.smk
from pathlib import Path

sys.path.append('../')
from scripts import gmet_to_summa_utils as gts_utils
from scripts import metsim_utils as ms_utils

# Resolve paths from the configuration file
config = gts_utils.resolve_paths(config)

# Create a list of the forcing files produced in the last workflow
input_forcing_list = gts_utils.list_files_in_subdirectory(config['easymore_output_dir'])
ensemble_list, file_path_list = gts_utils.build_ensemble_list(config['gmet_forcing_dir'])

easymore_output = Path(config['easymore_output_dir']) 
metsim_input = Path(config['metsim_input_dir'])

# Main rule to define the files produced by this workflow
rule prepare_metsim_files:
    input:
        Path(config["metsim_dir"], config["metsim_domain_nc"]),
        expand(Path(metsim_input,"{forcing}.nc"), forcing = input_forcing_list),
        expand(Path(metsim_input,"{forcing}_state.nc"), forcing = input_forcing_list)
         
# Create metsim domain file from an existing summa attribute file
rule create_metsim_domain_summa_attr:
    input:
        attr_nc = Path(config["attribute_nc"])
    output:
        domain_nc = Path(config["summa_dir"], config["metsim_domain_nc"])
    shell:
        'ncap2 -O -s "mask=elevation*0+1" {input.attr_nc} {output.domain_nc}'

rule subset_metsim_domain_to_forcing:
    input:
        domain_nc = Path(config["summa_dir"], config["metsim_domain_nc"]),
        input_forcing_files = expand(Path(config['easymore_output_dir'],"{file}.nc"), file=file_path_list)
    output:
        subset_nc = Path(config["metsim_dir"], config["metsim_domain_nc"])
    run:
        ms_utils.subset_domain_to_forcing(input.domain_nc, input.input_forcing_files[0], output.subset_nc)

# Define rule to run file remapping when remap file exists
rule prep_forcing_files_with_hru_id:
    input:
        input_forcing = Path(easymore_output,"{forcing}.nc")
    output:
        hru_id_temp = temp(Path(easymore_output,"{forcing}_hruId.nc")),
        hru_id = Path(metsim_input,"{forcing}.nc")
    group:
        "gmet_to_summa"
    shell:
        """
        ncap2 -O -s "hru=array(0,1,hruId)" {input.input_forcing} {output.hru_id_temp};
        ncatted -O -a long_name,hru,a,c,"hru coordinate index" {output.hru_id_temp} {output.hru_id}
        """
        
rule create_state_file:
    input:
        input_forcing_file = Path(metsim_input,"{forcing}.nc")
    output:
        output_state_file = Path(metsim_input,"{forcing}_state_temp.nc")
    group:
        "gmet_to_summa"
    run:
        ms_utils.create_state_file(input.input_forcing_file, output.output_state_file)

rule update_state_file_time:
    input:
        input_state_file = Path(metsim_input,"{forcing}_state_temp.nc")
    output:
        output_state_file = Path(metsim_input,"{forcing}_state.nc")
    group:
        "gmet_to_summa"
    run:
        gts_utils.update_time_units(input.input_state_file, output.output_state_file)


Overwriting ../rules/metsim_file_prep.smk


In [26]:
! snakemake --unlock -s ../rules/metsim_file_prep.smk  --configfile ../config/gmet_to_summa_test_case_config.yaml
! snakemake -s ../rules/metsim_file_prep.smk -c 8 --configfile ../config/gmet_to_summa_test_case_config.yaml

Unlocking working directory.
Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 8
Rules claiming more threads will be scaled down.
Job stats:
job                                count    min threads    max threads
-------------------------------  -------  -------------  -------------
create_metsim_domain_summa_attr        1              1              1
create_state_file                      3              1              1
prep_forcing_files_with_hru_id         3              1              1
prepare_metsim_files                   1              1              1
subset_metsim_domain_to_forcing        1              1              1
update_state_file_time                 3              1              1
total                                 12              1              1

Select jobs to execute...

[Mon Aug 21 13:09:20 2023]
rule prep_forcing_files_with_hru_id:
    input: ../../test_data/working_dir/easymore/output/001/gmet.01d.2021.001.nc
    output: ../../test_data/working

## Run metsim ##

In [27]:
%%writefile ../rules/run_metsim.smk
from pathlib import Path

sys.path.append('../')
from scripts import gmet_to_summa_utils as gts_utils
from scripts import metsim_utils as ms_utils

# Resolve paths from the configuration file
config = gts_utils.resolve_paths(config)

# Generate list of forcing files to create wildcards
input_forcing_list = gts_utils.list_files_in_subdirectory(Path(config['easymore_output_dir']))

rule run_metsim:
    input:
        expand(Path(config['metsim_output_dir'],"{id}.nc"), id=input_forcing_list)

rule generate_metsim_output:
    input:
         metsim_input_forcing = Path(config['metsim_input_dir'],"{id}.nc"),
         metsim_input_state = Path(config['metsim_input_dir'],"{id}_state.nc"),
         metsim_input_domain = Path(config["metsim_dir"], config["metsim_domain_nc"])
    output:
        metsim_output_forcing = Path(config['metsim_output_dir'],"{id}.nc")
    group:
        "gmet_to_summa"
    run:
        ms = ms_utils.create_metsim_config(config, input.metsim_input_forcing,input.metsim_input_state,output.metsim_output_forcing)
        ms.run()
        ms_utils.rename_metsim_output(ms,output.metsim_output_forcing)



Overwriting ../rules/run_metsim.smk


In [28]:
! snakemake --unlock -s ../rules/run_metsim.smk  --configfile ../config/gmet_to_summa_test_case_config.yaml
! snakemake -s ../rules/run_metsim.smk -c 8 --configfile ../config/gmet_to_summa_test_case_config.yaml

Unlocking working directory.
Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 8
Rules claiming more threads will be scaled down.
Job stats:
job                       count    min threads    max threads
----------------------  -------  -------------  -------------
generate_metsim_output        3              1              1
run_metsim                    1              1              1
total                         4              1              1

Select jobs to execute...

[Mon Aug 21 13:09:28 2023]
rule generate_metsim_output:
    input: ../../test_data/working_dir/metsim/input/002/gmet.01d.2021.002.nc, ../../test_data/working_dir/metsim/input/002/gmet.01d.2021.002_state.nc, ../../test_data/working_dir/metsim/domain.nc
    output: ../../test_data/working_dir/metsim/output/002/gmet.01d.2021.002.nc
    jobid: 3
    reason: Missing output files: ../../test_data/working_dir/metsim/output/002/gmet.01d.2021.002.nc
    wildcards: id=002/gmet.01d.2021.002
    resources: tmpdir=/

## Convert metsim output to summa ready input ##

In [29]:
%%writefile ../rules/metsim_to_summa.smk
from pathlib import Path

# Import custom functions
sys.path.append('../')
from scripts import gmet_to_summa_utils as gts_utils

# Resolve paths from the configuration file
config = gts_utils.resolve_paths(config)

input_forcing_list = gts_utils.list_files_in_subdirectory(config['metsim_output_dir'], '.nc')

rule metsim_to_summa:
    input:
        expand(Path(config['summa_forcing_dir'],"{id}.nc"), id=input_forcing_list)

rule create_hru_id_file:
    input:
        subset_domain_file = Path(config["metsim_dir"], config["metsim_domain_nc"])
    output:
        hru_id_file = temp(Path(config["metsim_dir"], 'hruId.nc'))
    shell:
        'ncks -v hruId {input.subset_domain_file} {output.hru_id_file}'

rule append_hru_id_and_datastep_to_metsim_output:
    input:
        input_metsim_file = Path(config['metsim_output_dir'],"{id}.nc"),
        hru_id_file = Path(config["metsim_dir"], 'hruId.nc')
    output:
        output_metsim_file_temp = temp(Path(config['summa_forcing_dir'],"{id}_temp.nc")),
        output_metsim_file = Path(config['summa_forcing_dir'],"{id}.nc")
    params:
        timestep = int(config["metsim_timestep_minutes"]) * 60
    shell:
        """
        if ! ncdump -h {input.input_metsim_file} | grep -q "hruId"; then
            ncks -h -A {input.hru_id_file} {input.input_metsim_file}
        fi
        ncks -O -C -x -v hru {input.input_metsim_file} {output.output_metsim_file_temp}
        ncrename -O -v .SWradAtm,SWRadAtm {output.output_metsim_file_temp}
        ncrename -O -v .LWradAtm,LWRadAtm {output.output_metsim_file_temp}
        ncap2 -s "data_step={params.timestep}" {output.output_metsim_file_temp} --append {output.output_metsim_file}
        """

        

Overwriting ../rules/metsim_to_summa.smk


In [30]:
! snakemake --unlock -s ../rules/metsim_to_summa.smk  --configfile ../config/gmet_to_summa_test_case_config.yaml
! snakemake -s ../rules/metsim_to_summa.smk -c 8 --configfile ../config/gmet_to_summa_test_case_config.yaml

Unlocking working directory.
Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 8
Rules claiming more threads will be scaled down.
Job stats:
job                                            count    min threads    max threads
-------------------------------------------  -------  -------------  -------------
append_hru_id_and_datastep_to_metsim_output        3              1              1
create_hru_id_file                                 1              1              1
metsim_to_summa                                    1              1              1
total                                              5              1              1

Select jobs to execute...

[Mon Aug 21 13:09:47 2023]
rule create_hru_id_file:
    input: ../../test_data/working_dir/metsim/domain.nc
    output: ../../test_data/working_dir/metsim/hruId.nc
    jobid: 2
    reason: Missing output files: ../../test_data/working_dir/metsim/hruId.nc
    resources: tmpdir=/var/folders/cf/h3xm_pj57nxdnm6lk83rzxt9jt0